In [1]:
%load_ext autoreload 
%autoreload 2
%reload_ext autoreload

import xarray as xr
import numpy as np
import os
import pandas as pd
from glob import glob
import datetime as dt
from PIL import Image
import sys
import functions as f
import pickle
import tensorflow as tf
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras import backend as K
from tensorflow.keras.utils import plot_model
from IPython.display import Image 
from tensorflow.python.data.ops import dataset_ops
import dask
from keras.models import load_model
from sklearn.metrics import mean_squared_error
from io import StringIO
from keras.layers import Input
import keras as k
from contextlib import redirect_stdout
from PIL import Image
import matplotlib.gridspec as gridspec
import channelExperiment as CE
import loadDataWeek0 as loadData
import bottleneck as bn
from tensorflow.keras import mixed_precision
import csv
import addPredictors as pred
import loadValues as lv
import masks
import denseValue
import xgboost as xgb
import json
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from concurrent.futures import ProcessPoolExecutor, as_completed
import bottleneck as bn
import concurrent.futures

global region_name
region_name = 'CONUS' #['australia','CONUS']


if region_name == 'CONUS':
    source = 'Data'
elif region_name == 'australia':
    source = 'Data_australia'


global RZSM_or_Tmax_or_both
RZSM_or_Tmax_or_both = 'RZSM' # for getting the predictor from either RZSM and Tmax ('both') or only RZSM ('RZSM')


global num_cores
#Select the number of cpu cores selected at start
num_cores = 10



2024-03-13 10:17:23.818520: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-13 10:17:28.256474: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-13 10:17:58.269396: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/glade/work/klesinger/conda-envs/tf212gpu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook imp

In [2]:
# import multiprocessing

# # Get the number of available CPU cores
# num_cores = multiprocessing.cpu_count()

# print(f"Number of available CPU cores: {num_cores}")


# Choose different loss and architecture configurations

In [3]:
# # #Load model and check summary
# model = load_model(f'checkpoints/Wk_1/Wk1_EX5',compile=False) #don't need the custom loss function for predictions
# model.summary() 


# This script will load data based on specific lead times that we want to experiment with and what Experiments we choose to run

# Week 1 (lead index 6 of the forecast)

| Ex. Number/Name | Obs. RZSM Inputs | Obs. Other Inputs                    | UNET predicted Inputs (N>=2)     |Reforecast Inputs| Prediction Lead Week/Variables | Activation function | Loss function | Batch size |
| ----------------| -----------------| ------------------------------------ | -------------------      |--------          | -------------------| ---------------------| ------------------------| ------------|
| 0 - EX0         | None             | None                                 | None                     | RZSM, Wk 1-N| Wk N     -  RZSM |              Relu             | CRPS experimental| 66
| 1 - EX1         | Wk. lags 1-3    | pwat,spfh,tmax,z200,diff_temp lags 1-3 | RZSM Wk 1-N |        None |  Wk N     -  RZSM |             Relu             | CRPS experimental | 66
| 2 - EX2         | Wk. lags 1-6   | pwat,spfh,tmax,z200,diff_temp lags 1-3  | RZSM Wk 1-N          |  None |  Wk N     -  RZSM |       Relu             | CRPS experimental| 66
| 3 - EX3         | Wk. lags 1-9    | pwat,spfh,tmax,z200,diff_temp lags 1-3 | RZSM Wk 1-N         |  None |  Wk N     -  RZSM |       Relu             | CRPS experimental| 66
| 4 - EX4         | Wk. lags 1-12    | pwat,spfh,tmax,z200,diff_temp lags 1-3| RZSM Wk 1-N         |  None |  Wk N     -  RZSM |      Relu             | CRPS experimental| 66
| 5 - EX5         | Wk. lags 1-3   | None                                   | None                   | RZSM, week N | Wk N     -  RZSM  |         Relu             | CRPS experimental| 66
| 6 - EX6         | Wk. lags 1-6   | None                                     | None                   |RZSM, week N  | Wk N     -  RZSM  |        Relu             | CRPS experimental| 66
| 7 - EX7         | Wk. lags 1-9    | None                                | None                   |RZSM, week N  | Wk N     -  RZSM  |         Relu             | CRPS experimental |66
| 8 - EX8         | Wk. lags 1-12    | None                                  | None                   |RZSM, week N  | Wk N     -  RZSM |         Relu             | CRPS experimental | 66
| 9 - EX9         | Wk. lags 1-3    |pwat,spfh,tmax,z200,diff_temp lags 1-3  | RZSM Wk 1-N                  |RZSM  week N  | Wk N     -  RZSM  |         Relu             | CRPS experimental| 66
| 10 - EX10       | Wk. lags 1-6    |pwat,spfh,tmax,z200,diff_temp  lags 1-3 | RZSM Wk 1-N                  |RZSM  week N  |Wk N     -  RZSM   |         Relu             | CRPS experimental| 66
| 11 - EX11       | Wk. lags 1-9    |pwat,spfh,tmax,z200,diff_temp  lags 1-3 | RZSM Wk 1-N                  |RZSM  week N  | Wk N     -  RZSM  |         Relu             | CRPS experimental| 66
| 12 - EX12       | Wk. lags 1-12   |pwat,spfh,tmax,z200,diff_temp lags 1-3  | RZSM Wk 1-N                  |RZSM  week N  | Wk N     -  RZSM  |         Relu             | CRPS experimental| 66
| 13 - EX13       | None             | None                                 | None                     | RZSM week N| Wk N     -  RZSM |              Relu             | CRPS experimental| 66
| 14 - EX14         | Wk. lags 1-3    | pwat,spfh,tmax,z200,diff_temp lags 1-3 | None |        None |  Wk N     -  RZSM |             Relu             | CRPS experimental | 66
| 15 - EX15         | Wk. lags 1-6   | pwat,spfh,tmax,z200,diff_temp lags 1-3  | None             |  None |  Wk N     -  RZSM |       Relu             | CRPS experimental| 66
| 16 - EX16         | Wk. lags 1-9    | pwat,spfh,tmax,z200,diff_temp lags 1-3 | None            |  None|  Wk N     -  RZSM |       Relu             | CRPS experimental| 66
| 17 - EX17        | Wk. lags 1-12    | pwat,spfh,tmax,z200,diff_temp lags 1-3| None            |  None|  Wk N     -  RZSM |      Relu             | CRPS experimental| 66
| 18 - EX18         | Wk. lags 1-3   | None                                   | RZSM Wk 1-N                 | RZSM, week N | Wk N     -  RZSM  |         Relu             | CRPS experimental| 66
| 19 - EX19         | Wk. lags 1-6   | None                                     | RZSM Wk 1-N                  |RZSM, week N  | Wk N     -  RZSM  |        Relu             | CRPS experimental| 66
| 20 - EX20         | Wk. lags 1-9    | None                                | RZSM Wk 1-N                  |RZSM, week N  | Wk N     -  RZSM  |         Relu             | CRPS experimental |66
| 21 - EX21         | Wk. lags 1-12    | None                                  | RZSM Wk 1-N                   |RZSM, week N  | Wk N     -  RZSM |         Relu             | CRPS experimental | 66
| 22 - EX22         | Wk. lags 1-3   | None                                   | None                   | None | Wk N     -  RZSM  |         Relu             | CRPS experimental| 66
| 23 - EX23         | Wk. lags 1-6   | None                                     | None                   |None  | Wk N     -  RZSM  |        Relu             | CRPS experimental| 66
| 24 - EX24         | Wk. lags 1-9    | None                                | None                   |None  | Wk N     -  RZSM  |         Relu             | CRPS experimental |66
| 25 - EX25         | Wk. lags 1-12    | None                                  | None                   |None  | Wk N     -  RZSM |         Relu             | CRPS experimental | 66
| 26 - EX26         | None    | None                                  | None                   |Wk 1-N (choose best models)  | Wk N     -  RZSM |         Relu             | CRPS experimental | 66
| 27 - EX27         | Wk. lags 1-3    |pwat,spfh,tmax,z200,diff_temp lags 1-3  |  None                   |Wk1 RZSM,tmax, diff_temp, z200, pwat, spfh  | Wk N     -  RZSM  |         Relu             | CRPS experimental| 66 (like EX9)
| 28 - EX28         | Wk. lags 1-6    |pwat,spfh,tmax,z200,diff_temp lags 1-3  |  None                   |Wk1 RZSM,tmax, diff_temp, z200, pwat, spfh  | Wk N     -  RZSM  |         Relu             | CRPS experimental| 66 (like EX10)

# EX 1-4 - Testing if the prediction of previous week adds value (but don't include the current week that is trying to be predicted). Observation driven but with a prediction.
# EX 14-17 - Testing if the prediction of previous week adds value (but don't include the current week that is trying to be predicted). Purely observation driven.
# EX 5-8 - Testing if adding other observations (pwat, spfh, etc) has an increase or decrease in skill.
# EX 18-21 - Seeing if adding the previous week gains additional skill
# EX 22-25 - Purely observation driven. Only soil moisture.

In [4]:
input_directory = f'Data/model_npy_inputs/{region_name}/Model_input_data' #For inputs into the model
verification_directory = f'Data/model_npy_inputs/{region_name}/Verification_data' #For observation verification
start_of_forecast_init = '2000-01-05'
end_of_forecast_init = '2019-12-25'

global training_size_shape
training_size_shape = np.array((9185,48,96))

global validation_testing_size_shape
validation_testing_size_shape = np.array((1144,48,96))

'''This decides how many lag weeks we have for data such as pwat, z200, tmax, etc'''
global observation_lag_list_not_RZSM
#This is for observations pwat, z200, spfh, tmax, diff_temp variables used as predictors
#These are the day lags which were already computed as the 7-day rolling mean

observation_lag_list_not_RZSM = [-1,-7,-14] 



In [5]:
def load_all_data_EX0(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, include_reforecast_or_not, observation_lag_list_not_RZSM, lag_integer_list, input_directory,training_size_shape,validation_testing_size_shape,RZSM_or_Tmax_or_both,addtl_experiment,experiment_test):

    soil_var = 'soilw_bgrnd' #This is the observation verification variable name
    
    channel_list = []

    if addtl_experiment == False:
        assert lead >=1, 'Lead must be >=1, do not look at Week 0'
        
        lead_list = np.arange(1,lead+1)

        print(f'Only adding Reforecast data as input for leads {list(lead_list)}.')

        training_input = np.empty(shape = (training_size_shape[0],training_size_shape[1],training_size_shape[2],lead)) #We are adding both RZSM and/or Tmax (so use * 2)
        validation_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],lead))
        testing_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],lead))

        #Add reforecast data
        for idx,lead in enumerate(lead_list):
            channel_list.append(f'RZSM_ref_lead{lead}')

            training_input, validation_input, testing_input = pred.add_reforecast_by_lag(training_input, validation_input, testing_input, lead, input_directory,soil_var, idx)

        print(f'Index idx value is {idx}. Done adding RZSM obs.')    

    
    elif addtl_experiment == True:
        print(f'Only adding Reforecast data as input for lead {lead}.')

        lead_add = 1 #We are only adding the current week of reforecast
        lead_list = [lead]

        training_input = np.empty(shape = (training_size_shape[0],training_size_shape[1],training_size_shape[2],lead_add)) #We are adding both RZSM and Tmax (so use * 2)
        validation_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],lead_add))
        testing_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],lead_add))

        channel_list.append(f'RZSM_ref_lead{lead}')

        idx = 0
        training_input, validation_input, testing_input = pred.add_reforecast_by_lag(training_input, validation_input, testing_input, lead, input_directory,soil_var, idx)

    return(training_input, validation_input, testing_input, channel_list)

In [6]:
# def load_all_data_EX1_EX2_EX3_EX4_after_week_0(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, include_reforecast_or_not, 
#                                            observation_lag_list_not_RZSM, lag_integer_list, input_directory,training_size_shape,
#                                            validation_testing_size_shape,experiment_name,RZSM_or_Tmax_or_both,addtl_experiment,experiment_test, experiment_name_out):
#     channel_list = []
    
#     #Reforecast predictors
#     var_list = ['pwat_eatm', 'spfh_2m', 'tmax_2m', 'diff_temp_2m', 'hgt_pres']

#     assert lead >=1, 'Lead must be >=1, we are not testing week 0 anymore'

#     if addtl_experiment == False:
#         #Get the number of extra channels to add from previous weeks predictions
#         if RZSM_or_Tmax_or_both == 'both':
#             add_channels = lead * 2
#         else:
#             #Also add the number of channels according to the lead. If lead == 1, then only add lead 1 current week. If lead == 2, add one channel for week 2 and one channel for the prediction from Week 1
#             if lead > 1:
#                 add_channels = lead - 1
#             else:
#                 add_channels = 0



#         training_input = np.empty(shape = (training_size_shape[0],training_size_shape[1],training_size_shape[2],len(lag_integer_list)+len(var_list)*3+add_channels))
#         validation_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],len(lag_integer_list)+len(var_list)*3+add_channels))
#         testing_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],len(lag_integer_list)+len(var_list)*3+add_channels))

#         print(f'Training shape input is {training_input.shape}')

#         #Add RZSM observations first
#         for idx,lag in enumerate(lag_integer_list):
#             channel_list.append(f'RZSM_obs_lag{lag}')
#             training_input, validation_input, testing_input = pred.add_obs_RZSM_by_lag(training_input, validation_input, testing_input, lag, input_directory, idx)

#         print(f'Index idx value is {idx}. Done adding RZSM obs.')
        
#         RZSM_train_obs, RZSM_validation_obs = pred.return_masking_objects_for_RZSM(input_directory)

        
#         for variable in var_list:
#             channel_name_output = pred.return_channel_name(variable)
#             for lag in observation_lag_list_not_RZSM:
#                 channel_list.append(f'{channel_name_output}_obs_lag{lag}')
#                 idx+=1
#                 #Observations adding
#                 training_input, validation_input, testing_input = pred.add_obs_other_observations_by_lag(training_input, validation_input, testing_input, lag, input_directory, variable, idx)
     
        
#             print(f'Index idx value is {idx}. Done adding {variable} obs.')
    
#         if lead in [0,1]:
#             pass
#         else:
#             #Now we need to load our data from the previous weeks
#             for lead_previous in range(1,lead):
#                 print(f'Working on previous lead: Week {lead_previous}')
#                 # break
    
#                 pred_dir = f'predictions_XGBOOST/{region_name}/Wk{lead_previous}_training_validation'
#                 testing_dir = f'predictions_XGBOOST/{region_name}/Wk{lead_previous}_testing'
    
#                 os.system(f'mkdir -p {pred_dir}')
    
#                 training_prediction_name= f'{pred_dir}/Wk{lead_previous}_training_{experiment_name_out}.npy'
#                 validation_prediction_name = f'{pred_dir}/Wk{lead_previous}_validation_{experiment_name_out}.npy'
#                 testing_prediction_name = f'{testing_dir}/Wk{lead_previous}_testing_{experiment_name_out}.npy'
    
#                 testing_prediction = np.load(testing_prediction_name)
                

#                 print(f'Loading data from previous week lead {lead_previous}')
#                 training_prediction = np.load(training_prediction_name)
#                 validation_prediction =np.load(validation_prediction_name)

#                 else:
#                     print(f'Creating prediction data from previous week lead {lead_previous}')
#                     '''If they don't exist..... load model for previous week'''
#                     model = load_model(f'checkpoints/Wk{lead_previous}/Wk{lead_previous}_{experiment_name_out}',compile=False) #don't need the custom loss function for predictions
    
#                     training_input_previous = np.load(f'Data/model_npy_inputs/{region_name}/Wk{lead_previous}_EX_input_data/{experiment_name_out}_training_input.npy')
#                     validation_input_previous = np.load(f'Data/model_npy_inputs/{region_name}/Wk{lead_previous}_EX_input_data/{experiment_name_out}_validation_input.npy')
#                     testing_input_previous = np.load(f'Data/model_npy_inputs/{region_name}/Wk{lead_previous}_EX_input_data/{experiment_name_out}_testing_input.npy')
    
#                     '''Make predictions and save data to load for previous use'''
#                     training_prediction = model.predict(training_input_previous) #Just for unit checking
#                     np.save(training_prediction_name,training_prediction)
    
#                     validation_prediction = model.predict(validation_input_previous)
#                     np.save(validation_prediction_name,validation_prediction)
    
#                 ############# ADD BACK TO CREATED ARRAY ###############################
#                 '''Now get the best UNET index prediction for RZSM and Tmax. Because the UNET plus plus produces 4 different outputs'''
#                 min_RZSM_index,min_tmax_index = f.load_loss_csv_bias_correction(lead_previous,experiment_name_out,RZSM_or_Tmax_or_both,region_name)
    
#                 if np.array(training_prediction).shape[0] == 8 and RZSM_or_Tmax_or_both == 'both':
#                     min_tmax_index = min_tmax_index+4 #Because index values 0-3 are RZSM and 4-7 are Tmax
#                 elif np.array(training_prediction).shape[0] == 6 and RZSM_or_Tmax_or_both == 'both':
#                     min_tmax_index = min_tmax_index + 3 #Because index values are 0-2 for RZSM and 3-5 for Tmax
#                 '''Now split the training and validation data back into RZSM and/or Tmax'''
    
    
#                 train_RZSM = np.array(training_prediction)[min_RZSM_index,:,:,:,:]
#                 val_RZSM = np.array(validation_prediction)[min_RZSM_index,:,:,:,:]
#                 test_RZSM = np.array(testing_prediction)[min_RZSM_index,:,:,:,:]

    
#                 '''Make sure we mask RZSM values which aren't on land'''
#                 train_RZSM = np.where(RZSM_train_obs==0,0,train_RZSM)
#                 val_RZSM = np.where(RZSM_validation_obs==0,0,val_RZSM)
#                 test_RZSM = np.where(RZSM_validation_obs==0,0,test_RZSM)
    
#                 #Now add back to the data
#                 '''Now add back to the newly created dataset'''
#                 idx+=1
#                 channel_list.append(f'RZSM_prediction_lead{lead_previous}')
#                 print(f'Adding RZSM training, validation, testing into index {idx}')
#                 training_input[:,:,:,idx] = train_RZSM[:,:,:,0]
#                 validation_input[:,:,:,idx] = val_RZSM[:,:,:,0]
#                 testing_input[:,:,:,idx] = test_RZSM[:,:,:,0]
    
#             if RZSM_or_Tmax_or_both == 'both':
#                 channel_list.append(f'tmax_prediction_lead{lead_previous}')
#                 idx+=1
#                 print(f'Adding Tmax training, validation, testing into index {idx}')
#                 training_input[:,:,:,idx] = train_tmax[:,:,:,0]
#                 validation_input[:,:,:,idx] = val_tmax[:,:,:,0]
#                 testing_input[:,:,:,idx] = test_tmax[:,:,:,0]
    

#             print('Final channel of input is the following (just make sure it has zeros for RZSM)')
#             print(training_input[0,:,:,-1])

#     elif addtl_experiment == True:
#         #Get the number of extra channels to add from previous weeks predictions
#         add_channels = 0


#         training_input = np.empty(shape = (training_size_shape[0],training_size_shape[1],training_size_shape[2],len(lag_integer_list)+len(observation_lag_list_not_RZSM)*5+add_channels))
#         validation_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],len(lag_integer_list)+len(observation_lag_list_not_RZSM)*5+add_channels))
#         testing_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],len(lag_integer_list)+len(observation_lag_list_not_RZSM)*5+add_channels))


#         #Add RZSM observations first
#         for idx,lag in enumerate(lag_integer_list):
#             channel_list.append(f'RZSM_obs_lag{lag}')
#             training_input, validation_input, testing_input = pred.add_obs_RZSM_by_lag(training_input, validation_input, testing_input, lag, input_directory, idx)

#         print(f'Index idx value is {idx}. Done adding RZSM obs.')    
        
#         RZSM_train_obs, RZSM_validation_obs = pred.return_masking_objects_for_RZSM(input_directory)


#         for variable in var_list:
#             channel_name_output = pred.return_channel_name(variable)
#             for lag in observation_lag_list_not_RZSM:
#                 channel_list.append(f'{channel_name_output}_obs_lag{lag}')
#                 idx+=1
#                 #Observations adding
#                 training_input, validation_input, testing_input = pred.add_obs_other_observations_by_lag(training_input, validation_input, testing_input, lag, input_directory, variable, idx)
     
        
#             print(f'Index idx value is {idx}. Done adding {variable} obs.')

#         print('Final channel of input is the following (just make sure it has zeros for RZSM)')
#         print(training_input[0,:,:,-1])

#     return(training_input, validation_input, testing_input,channel_list)

In [7]:
def load_all_data_EX5_EX6_EX7_EX8_experiments(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, 
                                              include_reforecast_or_not, observation_lag_list_not_RZSM, lag_integer_list,
                                              input_directory,training_size_shape,validation_testing_size_shape,RZSM_or_Tmax_or_both,experiment_test,experiment_name,experiment_name_out):
    print('\nUsing observations as predictions and forecast lead week 1\n')
    #Observations RZSM
    '''We need to combine all the RZSM files into 1 single dictionary for later processing'''
    
    channel_list = []

    assert lead >=1, 'We are only looking at weekly leads 1-5, cannot do 0 lead'
    
    #Reforecast predictors
    var_list = ['pwat_eatm', 'spfh_2m', 'tmax_2m', 'diff_temp_2m', 'hgt_pres']
    
    print(f'Experiment name is {experiment_name_out}')
    
    if experiment_name in ['EX22','EX23','EX24','EX25']:
        #This is only a bias correction using observations (no reforecast input at all)
        lead_add = 0
    elif experiment_name in ['EX5','EX6','EX7','EX8']:
        #This is only a bias correction using observations and the current week of reforecast as inputs
        lead_add = 1
    elif experiment_name in ['EX18','EX19','EX20','EX21']:
        if lead ==1:
            lead_add = 1 #Only adding the current week of forecast
        else:
            lead_add = lead #Adding previous weeks forecast and the current week

    print(f'Adding {lead_add} to channel list from reforecast.')

        
    training_input = np.empty(shape = (training_size_shape[0],training_size_shape[1],training_size_shape[2],len(lag_integer_list)+ lead_add))
    validation_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],len(lag_integer_list) + lead_add))
    testing_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],len(lag_integer_list)+ lead_add))
    
    print(f'Training shape: {training_input.shape}')
    
    #Add RZSM observations first
    for idx,lag in enumerate(lag_integer_list):
        channel_list.append(f'RZSM_obs_lag{lag}')
        training_input, validation_input, testing_input = pred.add_obs_RZSM_by_lag(training_input, validation_input, testing_input, lag, input_directory, idx)

    print(f'Index idx value is {idx}. Done adding RZSM obs.')
    
    #For masking
    RZSM_train_obs, RZSM_validation_obs = pred.return_masking_objects_for_RZSM(input_directory)

    if experiment_name in ['EX5','EX6','EX7','EX8']:
        #We only need to add the current reforecast week
        #Add final predictor for the current week of RZSM within reforecast
        idx+=1
        soil_var = 'soilw_bgrnd'
        channel_name_output = pred.return_channel_name(soil_var)
        channel_list.append(f'{channel_name_output}_ref_lead{lead}')
    
        print(f'And finally adding RZSM week {lead} from reforecast as a predictor')
        
        training_input, validation_input, testing_input = pred.add_reforecast_by_lag(training_input, validation_input, testing_input, lead, input_directory,soil_var, idx)
        
        return(training_input, validation_input, testing_input, channel_list)

    elif experiment_name in ['EX22','EX23','EX24','EX25']:
         #We are not adding any additional predictors
         return(training_input, validation_input, testing_input, channel_list)
        
    
    else:
        assert lead >= 2, 'If trying to run EX18-EX21, DO NOT RUN with week 1 data. Start with Week 2'



        #Now we need to load our data from the previous weeks
        for lead_previous in range(1,lead):

            #To not have to re-run a bunch more models, we can re-use the training from a previous week
            if lead_previous == 1:
                if experiment_name == 'EX18':
                    prev_experiment = f'EX5_{save_loss_name}_RZSM'
                        
                elif experiment_name == 'EX19':
                    prev_experiment = f'EX6_{save_loss_name}_RZSM'    
                    
                elif experiment_name == 'EX20':
                    prev_experiment = f'EX7_{save_loss_name}_RZSM'  
                    
                elif experiment_name == 'EX21':
                    prev_experiment = f'EX8_{save_loss_name}_RZSM'
            elif lead > 2:
                prev_experiment = experiment_name_out
            
            print(f'Working on previous lead: Week {lead_previous}')
            # break

            pred_dir = f'predictions_XGBOOST/{region_name}/Wk{lead_previous}_training_validation'
            testing_dir = f'predictions_XGBOOST/{region_name}/Wk{lead_previous}_testing'

            os.system(f'mkdir -p {pred_dir}')

            training_prediction_name= f'{pred_dir}/Wk{lead_previous}_training_{prev_experiment}.npy'
            validation_prediction_name = f'{pred_dir}/Wk{lead_previous}_validation_{prev_experiment}.npy'
            testing_prediction_name = f'{testing_dir}/Wk{lead_previous}_testing_{prev_experiment}.npy'


            testing_prediction = np.load(testing_prediction_name)
            '''Load the training and validation predictions if they exist'''
            
            if os.path.exists(training_prediction_name):
                print(f'Loading data from previous week lead {lead_previous}')
                training_prediction = np.load(training_prediction_name)
                validation_prediction =np.load(validation_prediction_name)

            else:
                print(f'Creating prediction data from previous week lead {lead_previous} from {prev_experiment}.')
                '''If they don't exist..... load model for previous week'''
                model = load_model(f'checkpoints/Wk_{lead_previous}/Wk{lead_previous}_{prev_experiment}',compile=False) #don't need the custom loss function for predictions

                training_input_previous = np.load(f'Data/model_npy_inputs/{region_name}/Wk{lead_previous}_EX_input_data/{prev_experiment}_training_input.npy')
                validation_input_previous = np.load(f'Data/model_npy_inputs/{region_name}/Wk{lead_previous}_EX_input_data/{prev_experiment}_validation_input.npy')
                testing_input_previous = np.load(f'Data/model_npy_inputs/{region_name}/Wk{lead_previous}_EX_input_data/{prev_experiment}_testing_input.npy')

                '''Make predictions and save data to load for previous use'''
                training_prediction = model.predict(training_input_previous) #Just for unit checking
                np.save(training_prediction_name,training_prediction)

                validation_prediction = model.predict(validation_input_previous)
                np.save(validation_prediction_name,validation_prediction)

            ############# ADD BACK TO CREATED ARRAY ###############################
            '''Now get the best UNET index prediction for RZSM and Tmax. Because the UNET plus plus produces 4 different outputs'''
            min_RZSM_index,min_tmax_index = f.load_loss_csv_bias_correction(lead_previous,prev_experiment,RZSM_or_Tmax_or_both, region_name)

            if np.array(training_prediction).shape[0] == 8 and RZSM_or_Tmax_or_both == 'both':
                min_tmax_index = min_tmax_index+4 #Because index values 0-3 are RZSM and 4-7 are Tmax
            elif np.array(training_prediction).shape[0] == 6 and RZSM_or_Tmax_or_both == 'both':
                min_tmax_index = min_tmax_index + 3 #Because index values are 0-2 for RZSM and 3-5 for Tmax
            '''Now split the training and validation data back into RZSM and/or Tmax'''


            train_RZSM = np.array(training_prediction)[min_RZSM_index,:,:,:,:]
            val_RZSM = np.array(validation_prediction)[min_RZSM_index,:,:,:,:]
            test_RZSM = np.array(testing_prediction)[min_RZSM_index,:,:,:,:]


            '''Make sure we mask RZSM values which aren't on land'''
            train_RZSM = np.where(RZSM_train_obs==0,0,train_RZSM)
            val_RZSM = np.where(RZSM_validation_obs==0,0,val_RZSM)
            test_RZSM = np.where(RZSM_validation_obs==0,0,test_RZSM)

            #Now add back to the data
            '''Now add back to the newly created dataset'''
            idx+=1
            channel_list.append(f'RZSM_prediction_lead{lead_previous}')
            print(f'Adding RZSM training, validation, testing into index {idx}')
            training_input[:,:,:,idx] = train_RZSM[:,:,:,0]
            validation_input[:,:,:,idx] = val_RZSM[:,:,:,0]
            testing_input[:,:,:,idx] = test_RZSM[:,:,:,0]


        #Add final predictor for the current week of RZSM within reforecast
        idx+=1
        soil_var = 'soilw_bgrnd'
        channel_name_output = pred.return_channel_name(soil_var)
        channel_list.append(f'{channel_name_output}_ref_lead{lead}')
    
        print(f'And finally adding RZSM week {lead} from reforecast as a predictor')
        
        training_input, validation_input, testing_input = pred.add_reforecast_by_lag(training_input, validation_input, testing_input, lead, input_directory,soil_var, idx)
    
        print(f'Index idx value is {idx}. Done adding {soil_var} reforecast.')

        print('Final channel of input is the following (just make sure it has zeros for RZSM)')
        print(training_input[0,:,:,-1])


        return(training_input, validation_input, testing_input, channel_list)

In [8]:
def load_all_data_EX9_EX10_EX11_EX12_after_week_0(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, include_reforecast_or_not, 
                                           observation_lag_list_not_RZSM, lag_integer_list, input_directory,training_size_shape,
                                           validation_testing_size_shape,experiment_name,RZSM_or_Tmax_or_both, experiment_test,region_name,experiment_name_out):
    '''We need to combine all the RZSM files and all the observation data (pwat, spfh, tmax, diff_temp, z200) into one file. Also adding the RZSM reforecast data from RZSM and Tmax '''

    channel_list = []
    
    #Reforecast predictors
    var_list = ['pwat_eatm', 'spfh_2m', 'tmax_2m', 'diff_temp_2m', 'hgt_pres']

    if experiment_test == 0:
        add_channels = lead  
        reforecast_predictors = False
    elif (experiment_test >= 1) and (lead in [1,2]):
        #This is adding 5 additional predictors (tmax, diff_temp, z200, pwat, spfh)
        #Also add the number of channels according to the lead. If lead == 1, then only add lead 1 current week. If lead == 2, add one channel for week 2 and one channel for the prediction from Week 1
        add_channels = lead  + len(var_list)
        reforecast_predictors = True
    elif (experiment_test >= 1) and (lead > 2):
        add_channels = lead 
        reforecast_predictors = False
            
   
    training_input = np.empty(shape = (training_size_shape[0],training_size_shape[1],training_size_shape[2],len(lag_integer_list)+len(var_list)*3+add_channels))
    validation_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],len(lag_integer_list)+len(var_list)*3+add_channels))
    testing_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],len(lag_integer_list)+len(var_list)*3+add_channels))
    
    print(f'Training input shape = {training_input.shape}')
    
    #Add RZSM observations first
    for idx,lag in enumerate(lag_integer_list):
        channel_list.append(f'RZSM_obs_lag{lag}')
        training_input, validation_input, testing_input = pred.add_obs_RZSM_by_lag(training_input, validation_input, testing_input, lag, input_directory, idx)
        
    print(f'Index idx value is {idx}. Done adding RZSM obs.')    
    
    RZSM_train_obs, RZSM_validation_obs = pred.return_masking_objects_for_RZSM(input_directory)

    for variable in var_list:
        channel_name_output = pred.return_channel_name(variable)
        for lag in observation_lag_list_not_RZSM:
            channel_list.append(f'{channel_name_output}_obs_lag{lag}')
            idx+=1
            #Observations adding
            training_input, validation_input, testing_input = pred.add_obs_other_observations_by_lag(training_input, validation_input, testing_input, lag, input_directory, variable, idx)
 
    
        print(f'Index idx value is {idx}. Done adding {variable} obs.')
    
    ############################### NOW ADD THE PREDICTION DATA FROM PREVIOUS WEEK ################################################
    #Now we need to load our data from the previous weeks 
    
    if lead in [0, 1]:
        #DO NOT INCLUDE LEAD 0 AS A PREDICTOR (FROM PREVIOUS WEEK)
        pass
    else:

        for lead_previous in range(1,lead):
            print(f'Adding previous RZSM prediction from week {lead_previous} as an input channel')
            # break
            
            pred_dir = f'predictions_XGBOOST/{region_name}/Wk{lead_previous}_training_validation'
            testing_dir = f'predictions_XGBOOST/{region_name}/Wk{lead_previous}_testing'
            
            os.system(f'mkdir -p {pred_dir}')
            
            training_prediction_name= f'{pred_dir}/Wk{lead_previous}_training_{experiment_name_out}.npy'
            validation_prediction_name = f'{pred_dir}/Wk{lead_previous}_validation_{experiment_name_out}.npy'
            testing_prediction_name = f'{testing_dir}/Wk{lead_previous}_testing_{experiment_name_out}.npy'
            
            test_RZSM = np.load(testing_prediction_name)
            #Need to change the dimensions to match the other inputs
            #test_RZSM is shape (104,48,96)
            #Need to change to (1144,48,96)

            new_shape_test = np.zeros(shape = (104,11,48,96))
            for i in range(11):
                new_shape_test[:,i,:,:] = test_RZSM
            new_shape_test = np.reshape(new_shape_test,(1144,48,96))
            
            print(f'Loading data from previous week lead {lead_previous}')
            
            train_RZSM = np.load(training_prediction_name)
            new_shape_train = np.zeros(shape = (835,11,48,96))
            for i in range(11):
                new_shape_train[:,i,:,:] = train_RZSM
            new_shape_train = np.reshape(new_shape_train,(9185,48,96))

            val_RZSM =np.load(validation_prediction_name)
            new_shape_val = np.zeros(shape = (104,11,48,96))
            for i in range(11):
                new_shape_val[:,i,:,:] = val_RZSM
            new_shape_val = np.reshape(new_shape_val,(1144,48,96))
            
    
            #Now add back to the data
            '''Now add back to the newly created dataset'''
            idx+=1
            channel_list.append(f'RZSM_prediction_lead{lead_previous}')
            print(f'Adding RZSM training, validation, testing into index {idx}')
            training_input[:,:,:,idx] = new_shape_train[:,:,:]
            validation_input[:,:,:,idx] = new_shape_val[:,:,:]
            testing_input[:,:,:,idx] = new_shape_test[:,:,:]

    
    ############################### NOW ADD THE REFORECAST DATA FROM THE WEEK TO BE PREDICTED ################################################
    
    #Add predictors from the reforecast model
    if (lead in [1,2]) and (experiment_test == 1):
        print(f'Adding current reforecast data from week {lead} for vars {(var_list)}')  
        #Only add these for week 1 and 2. 
        for variable in var_list:
            idx+=1
            channel_name_output = pred.return_channel_name(variable)
            
            channel_list.append(f'{channel_name_output}_ref_lead{lead}')
            
            training_input, validation_input, testing_input = pred.add_reforecast_by_lag(training_input, validation_input, testing_input, lead, input_directory, variable, idx)
    
            print(f'Index idx value is {idx}. Done adding {variable} reforecast.')

    #Add final predictor for the current week of RZSM within reforecast
    idx+=1
    soil_var = 'soilw_bgrnd'
    channel_name_output = pred.return_channel_name(soil_var)
    channel_list.append(f'{channel_name_output}_ref_lead{lead}')

    print(f'And finally adding RZSM week {lead} from reforecast as a predictor')
    
    training_input, validation_input, testing_input = pred.add_reforecast_by_lag(training_input, validation_input, testing_input, lead, input_directory,soil_var, idx)

    print(f'Index idx value is {idx}. Done adding {soil_var} reforecast.')

    print('Final channel of input is the following (just make sure it has zeros for RZSM)')
    print(training_input[0,:,:,-1])

    return(training_input, validation_input, testing_input,channel_list)




In [9]:
def load_only_predictions_EX26(lead, input_directory,training_size_shape,validation_testing_size_shape,RZSM_or_Tmax_or_both,experiment_name,region_name,
                               experiment_name_out):
    '''We need to combine all the RZSM files and all the observation data (pwat, spfh, tmax, diff_temp, z200) into one file. Also adding the RZSM reforecast data from RZSM and Tmax '''

    channel_list = []

    assert lead >=2, 'We cannot make a prediction if lead week is 1 because we dont have an input. So make lead week >= 2.'

    #Get the number of extra channels to add from previous weeks predictions
    add_channels = lead # We are only adding the previous weeks lag and the current week. 

    training_input = np.empty(shape = (training_size_shape[0],training_size_shape[1],training_size_shape[2],add_channels))
    validation_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],add_channels))
    testing_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],add_channels))
    
    print(f'Training input shape = {training_input.shape}')
    
    #For masking
    RZSM_train_obs, RZSM_validation_obs = pred.return_masking_objects_for_RZSM(input_directory)

    ############################### NOW ADD THE PREDICTION DATA FROM PREVIOUS WEEK ################################################
        #Now we need to load our data from the previous weeks
    for idx,lead_previous in enumerate(range(1,lead)):
        # break
        
        pred_dir = f'predictions/{region_name}/Wk{lead_previous}_training_validation'
        testing_dir = f'predictions/{region_name}/Wk{lead_previous}_testing'
        
        os.system(f'mkdir -p {pred_dir}')
        
        previous_model = return_fully_autoregressive_EX26(lead)
            
        
        training_prediction_name= f'{pred_dir}/Wk{lead_previous}_training_{previous_model[idx]}.npy'
        validation_prediction_name = f'{pred_dir}/Wk{lead_previous}_validation_{previous_model[idx]}.npy'
        testing_prediction_name = f'{testing_dir}/Wk{lead_previous}_testing_{previous_model[idx]}.npy'
        
        testing_prediction = np.load(testing_prediction_name)
        '''Load the training and validation predictions if they exist'''
        
        if os.path.exists(training_prediction_name):
            print(f'Loading data from previous week lead {lead_previous}')
            training_prediction = np.load(training_prediction_name)
            validation_prediction =np.load(validation_prediction_name)
            
        else:
            print(f'Creating prediction data from previous week lead {lead_previous}')
            '''If they don't exist..... load model for previous week'''
            model = load_model(f'checkpoints/Wk_{lead_previous}/Wk{lead_previous}_{experiment_name}',compile=False) #don't need the custom loss function for predictions
            
            training_input_previous = np.load(f'Data/model_npy_inputs/{region_name}/Wk{lead_previous}_EX_input_data/{experiment_name_out}_training_input.npy')
            validation_input_previous = np.load(f'Data/model_npy_inputs/{region_name}/Wk{lead_previous}_EX_input_data/{experiment_name_out}_validation_input.npy')
            testing_input_previous = np.load(f'Data/model_npy_inputs{region_name}//Wk{lead_previous}_EX_input_data/{experiment_name_out}_testing_input.npy')
            
            '''Make predictions and save data to load for previous use'''
            training_prediction = model.predict(training_input_previous) #Just for unit checking
            np.save(training_prediction_name,training_prediction)
            
            validation_prediction = model.predict(validation_input_previous)
            np.save(validation_prediction_name,validation_prediction)
        
        
        ############# ADD BACK TO CREATED ARRAY ###############################
        '''Now get the best UNET index prediction for RZSM and Tmax. Because the UNET plus plus produces 4 different outputs'''
        min_RZSM_index,min_tmax_index = f.load_loss_csv_bias_correction(lead_previous,previous_model[idx],RZSM_or_Tmax_or_both, region_name)

        if np.array(training_prediction).shape[0] == 8 and RZSM_or_Tmax_or_both == 'both':
            min_tmax_index = min_tmax_index+4 #Because index values 0-3 are RZSM and 4-7 are Tmax
        elif np.array(training_prediction).shape[0] == 6 and RZSM_or_Tmax_or_both == 'both':
            min_tmax_index = min_tmax_index + 3 #Because index values are 0-2 for RZSM and 3-5 for Tmax
        '''Now split the training and validation data back into RZSM and/or Tmax'''
        
        
        train_RZSM = np.array(training_prediction)[min_RZSM_index,:,:,:,:]
        val_RZSM = np.array(validation_prediction)[min_RZSM_index,:,:,:,:]
        test_RZSM = np.array(testing_prediction)[min_RZSM_index,:,:,:,:]


        '''Make sure we mask RZSM values which aren't on land'''
        train_RZSM = np.where(RZSM_train_obs==0,0,train_RZSM)
        val_RZSM = np.where(RZSM_validation_obs==0,0,val_RZSM)
        test_RZSM = np.where(RZSM_validation_obs==0,0,test_RZSM)

        #Now add back to the data
        '''Now add back to the newly created dataset'''
        channel_list.append(f'RZSM_prediction_lead{lead_previous}')
        print(f'Adding RZSM training, validation, testing into index {idx}')
        training_input[:,:,:,idx] = train_RZSM[:,:,:,0]
        validation_input[:,:,:,idx] = val_RZSM[:,:,:,0]
        testing_input[:,:,:,idx] = test_RZSM[:,:,:,0]

        idx+=1

    
    ############################### NOW ADD THE REFORECAST DATA FROM THE WEEK TO BE PREDICTED ################################################
    
    #Add RZSM reforecast week N
    soil_var = 'soilw_bgrnd'
    channel_name_output = pred.return_channel_name(soil_var)
    channel_list.append(f'{channel_name_output}_ref_lead{lead}')

    print(f'And finally adding RZSM week {lead} from reforecast as a predictor')
    
    training_input, validation_input, testing_input = pred.add_reforecast_by_lag(training_input, validation_input, testing_input, lead, input_directory,soil_var, idx)

    print(f'Index idx value is {idx}. Done adding {soil_var} reforecast.')

    print('Final channel of input is the following (just make sure it has zeros for RZSM)')
    print(training_input[0,:,:,-1])

    return(training_input, validation_input, testing_input,channel_list)




In [10]:
def make_UNET_stacked_inputs(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, include_reforecast_or_not, 
                             observation_lag_list_not_RZSM, lag_integer_list, input_directory,training_size_shape,
                             validation_testing_size_shape,experiment_name,RZSM_or_Tmax_or_both,addtl_experiment,experiment_test,region_name,experiment_name_out):
    
    
    #Set restrictions. For EX0, we only want to bias correct the forecasts with week lead 1 RZSM and tmax
    if experiment_name == 'EX0' or experiment_name == 'EX13':
        
        #Step 1 load data 
        print(f'\nWorking on setting up data for Experiment {experiment_name} for lead {lead}\n')
        
        final_input_train, final_input_validation, final_input_testing,channel_list =  loadData.load_all_data_EX0(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, include_reforecast_or_not, 
                                                                                                     observation_lag_list_not_RZSM, lag_integer_list, input_directory,training_size_shape,
                                                                                                     validation_testing_size_shape,RZSM_or_Tmax_or_both,addtl_experiment,experiment_test)

        print(f'\nInput channels will be only week 1 Tmax and RZSM through the current lead {lead} which are also going to be predicted. There are no observations in this experiment\n')
        
        print('Done')
        
        
    
    ############################################### EXPERIMENTS 1-4 ####################################################################################
    elif experiment_name in ['EX1','EX2','EX3','EX4','EX14','EX15','EX16','EX17']:

        print(f'\nWorking on setting up data for Experiment {experiment_name} for lead {lead}\n')
        
        if lead == 0:
            #Step 1 load data 
            final_input_train, final_input_validation, final_input_testing,channel_list = loadData.load_all_data_EX1_EX2_EX3_EX4(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, 
                                                                                                                    include_reforecast_or_not, observation_lag_list_not_RZSM, lag_integer_list, input_directory,
                                                                                                                   training_size_shape,validation_testing_size_shape,RZSM_or_Tmax_or_both,
                                                                                                                                   region_name=region_name)
        else:
            final_input_train, final_input_validation, final_input_testing,channel_list = load_all_data_EX1_EX2_EX3_EX4_after_week_0(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, include_reforecast_or_not, 
                                                                                                                                       observation_lag_list_not_RZSM, lag_integer_list, input_directory,training_size_shape,
                                                                                                                                       validation_testing_size_shape,experiment_name,RZSM_or_Tmax_or_both,addtl_experiment,experiment_test,
                                                                                                                                    experiment_name_out=experiment_name_out)

        
                
        print('Done')
        
        
    
    ############################################### EXPERIMENTS 5-8 ####################################################################################
    elif experiment_name in ['EX5','EX6','EX7','EX8','EX18','EX19','EX20','EX21','EX22','EX23','EX24','EX25']:

        print(f'\nWorking on setting up data for Experiment {experiment_name} for lead {lead}\n')
        
        # #Step 1 load data 
        # if addtl_experiment == False:
        #     final_input_train, final_input_validation, final_input_testing,channel_list = loadData.load_all_data_EX5_EX6_EX7_EX8(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, 
        #                                                                                                    include_reforecast_or_not, observation_lag_list_not_RZSM, lag_integer_list,
        #                                                                                                    input_directory,training_size_shape,validation_testing_size_shape,RZSM_or_Tmax_or_both,
                                                                                                                                   # region_name=region_name)
        # else:
        final_input_train, final_input_validation, final_input_testing,channel_list = load_all_data_EX5_EX6_EX7_EX8_experiments(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, include_reforecast_or_not, 
                                                                                      observation_lag_list_not_RZSM, lag_integer_list, input_directory,training_size_shape,
                                                                                      validation_testing_size_shape,RZSM_or_Tmax_or_both,experiment_test,experiment_name,experiment_name_out=experiment_name_out)

        print('Done')
        
        
    
    ############################################### EXPERIMENTS 9-12 ####################################################################################
    elif (experiment_name == 'EX9') or (experiment_name == 'EX10') or (experiment_name == 'EX11') or (experiment_name == 'EX12') or (experiment_name == 'EX27') or (experiment_name == 'EX28'):
        
        print(f'\nWorking on setting up data for Experiment {experiment_name} for lead {lead}\n')

        if lead == 0:
            final_input_train, final_input_validation, final_input_testing,channel_list = loadData.load_all_data_EX9_EX10_EX11_EX12(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, 
                                                                                                              include_reforecast_or_not, observation_lag_list_not_RZSM, lag_integer_list, 
                                                                                                              input_directory,training_size_shape,validation_testing_size_shape,RZSM_or_Tmax_or_both, 
                                                                                                                                    experiment_test = experiment_test,
                                                                                                                                   region_name=region_name)
        else:
            final_input_train, final_input_validation, final_input_testing,channel_list = load_all_data_EX9_EX10_EX11_EX12_after_week_0(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, include_reforecast_or_not, 
                                                                                                       observation_lag_list_not_RZSM, lag_integer_list, input_directory,training_size_shape,
                                                                                                   validation_testing_size_shape,experiment_name,RZSM_or_Tmax_or_both,
                                                                                                                                       experiment_test = experiment_test,
                                                                                                                                       region_name = region_name,experiment_name_out=experiment_name_out)
    elif (experiment_name == 'EX26'):

        print(f'\nWorking on setting up data for Experiment {experiment_name} for lead {lead}\n')

        final_input_train, final_input_validation, final_input_testing,
        channel_list = load_only_predictions_EX26(lead, input_directory,training_size_shape,validation_testing_size_shape,RZSM_or_Tmax_or_both,experiment_name,region_name,
                               experiment_name_out)
        print('Done')


    return (final_input_train,final_input_validation,final_input_testing,channel_list)


In [11]:
#To try and save on memory, only load certain files at a time
def return_only_train_validation(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, include_reforecast_or_not, 
                                 observation_lag_list_not_RZSM, lag_integer_list, input_directory,training_size_shape,
                                 validation_testing_size_shape,experiment_name):
    
    reforecast_train_input, reforecast_validation_input, reforecast_testing_input,channel_list \
    = make_UNET_stacked_inputs(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, 
                               include_reforecast_or_not, observation_lag_list_not_RZSM, 
                               lag_integer_list, input_directory,training_size_shape,validation_testing_size_shape,experiment_name,region_name)

    return(reforecast_train_input, reforecast_validation_input,reforecast_testing_input,channel_list)





def return_testing_data_model_training(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, 
                               include_reforecast_or_not, observation_lag_list_not_RZSM, 
                               lag_integer_list, input_directory,training_size_shape,validation_testing_size_shape,experiment_name):

    reforecast_train_input, reforecast_validation_input, reforecast_testing_input,channel_list = make_UNET_stacked_inputs(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, 
                               include_reforecast_or_not, observation_lag_list_not_RZSM, 
                               lag_integer_list, input_directory,training_size_shape,validation_testing_size_shape,experiment_name,region_name)
    return(reforecast_testing_input,channel_list)

In [12]:
def return_experiment_name_and_lags_and_channel_dir(include_reforecast_or_not, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, addtl_experiment, experiment_test, region_name):
    
    experiment_name = CE.return_experiment_name(include_reforecast_or_not,num_lags_obs_RZSM,include_lags_obs_pwat_spfh_tmax,addtl_experiment,experiment_test)

    save_experiment_name = f'{experiment_name}_RZSM'

    experiment_name_out = f'{experiment_name}_XGBOOST_RZSM' #Need to manually change this if using different loss functions to keep track of things
    
    global lag_integer_list
    lag_integer_list =  CE.return_num_day_lags_from_weekly_lags(num_lags_obs_RZSM) #For number of RZSM observation lags

    #Where to save channel information
    save_experiment_dir = f'Data/model_npy_inputs/{region_name}/Wk{lead}_EX_input_data'
    
    channel_save_dir = f'channel_list_information/Wk{lead}'
    
    checkpoint_filepath = f'checkpoints_XGBOOST/{region_name}/Wk{lead}/Wk{lead}_{experiment_name_out}'
    
    losses_dir = f'Losses_with_OBS_XGBOOST/{region_name}/Wk{lead}'
    
    save_checkpoint_dir =f'checkpoints_XGBOOST/{region_name}/Wk{lead}'
    
    os.system(f'mkdir -p {checkpoint_filepath} {channel_save_dir} {save_experiment_dir} {losses_dir} {save_checkpoint_dir}')

    
    #Set up files for either saving or loading
    training_input_file = f'{save_experiment_dir}/{save_experiment_name}_XGBOOST_training_input.npy'
    validation_input_file = f'{save_experiment_dir}/{save_experiment_name}_XGBOOST_validation_input.npy'
    testing_input_file = f'{save_experiment_dir}/{save_experiment_name}_XGBOOST_testing_input.npy'

    
    return(save_experiment_name,experiment_name_out,experiment_name,lag_integer_list,channel_save_dir,
           training_input_file,validation_input_file,testing_input_file,checkpoint_filepath,
          losses_dir,save_checkpoint_dir,save_experiment_dir)

In [13]:
def generate_model_inputs(region_name,lead,include_reforecast_or_not,num_lags_obs_RZSM,include_lags_obs_pwat_spfh_tmax,addtl_experiment,experiment_test):

    save_experiment_name, experiment_name_out, experiment_name,lag_integer_list, \
    channel_save_dir, training_input_file, validation_input_file, \
    testing_input_file,checkpoint_filepath, losses_dir,save_checkpoint_dir, \
    save_experiment_dir = return_experiment_name_and_lags_and_channel_dir(include_reforecast_or_not, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, addtl_experiment, experiment_test, region_name)

    if os.path.exists(training_input_file) and os.path.exists(validation_input_file) and os.path.exists(testing_input_file):
        pass
    else:
        reforecast_train_input, reforecast_validation_input, reforecast_testing_input, channel_list \
        = make_UNET_stacked_inputs(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, include_reforecast_or_not, 
                             observation_lag_list_not_RZSM, lag_integer_list, input_directory,training_size_shape,
                             validation_testing_size_shape,experiment_name,RZSM_or_Tmax_or_both,addtl_experiment,experiment_test,region_name,experiment_name_out)
    
        #Save data to file
        np.save(training_input_file,tf.convert_to_tensor(reforecast_train_input,dtype=tf.float32))
        np.save(validation_input_file,tf.convert_to_tensor(reforecast_validation_input,dtype=tf.float32))
        np.save(testing_input_file,tf.convert_to_tensor(reforecast_testing_input,dtype=tf.float32))
        
        image_size = reforecast_train_input.shape[1:]           
        #Save channel list information to txt file
    
        with open(f'{channel_save_dir}/{save_experiment_name}_channel_list.txt', 'w') as file:
            for idx,element in enumerate(channel_list):
                if '-' in element:
                    source_='OBSERVATIONS'
                else:
                    source_='REFORECAST'
    
                file.write(f'Channel_{idx} is from {source_} with lead or lag {str(element)}' + '\n')

    return(f'Completed writing lead {lead} model input data to {save_experiment_dir}.')

# Run the model for each experiment

In [14]:
'''For experiments, you need to run the script once to make the data. Then restart the kernel to load the data. 
Otherwise the memory leaks are too high'''
def run_EXPERIMENT(lead, num_lags_obs_RZSM,include_lags_obs_pwat_spfh_tmax ,include_reforecast_or_not,addtl_experiment,experiment_test,region_name):
    if lead_week == 5:
        patience=15
    else:
        patience = 10

    #Options for num_lags_obs_RZSM weekly lags [ 3,6,9,12 ] ** this inc
    #Options for num_lags_obs_pwat_spfh_tmax weekly lags [0 weeks or 2 weeks ]

    train(epochs = 150, 
          batch_size = 66, 
          lead=lead_week, 
          initial_learning_rate=0.0001,
          beta_1 = 0.9,
          shuffle=False,
          patience=patience,
          kernel_norm = None,
          deep_supervision = True,
          num_lags_obs_RZSM=num_lags_obs_RZSM, 
          include_lags_obs_pwat_spfh_tmax=include_lags_obs_pwat_spfh_tmax,
          include_reforecast_or_not=include_reforecast_or_not,
          number_of_UNET_backbone_max_pool=4,
          permutation_test = False,
          make_additional_predictions_from_model_for_testing = False,
          input_directory=input_directory,
          training_size_shape=training_size_shape,
          validation_testing_size_shape=validation_testing_size_shape,
          addtl_experiment = addtl_experiment,
          experiment_test =experiment_test,
          region_name = region_name)
    return(0)



In [15]:
#These are models which only include the reforecast predictions. We are not including any observations into them.

EX0 ={'region_name':region_name, 'num_lags_obs_RZSM': 0, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }
EX13={'region_name':region_name, 'num_lags_obs_RZSM': 0, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':1 }


In [16]:
#Include all predictors for obs but DO NOT include the current reforecast RZSM 

EX1 ={'region_name':region_name, 'num_lags_obs_RZSM': 3, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': False, 'addtl_experiment': False, 'experiment_test':0 }
EX2 ={'region_name':region_name, 'num_lags_obs_RZSM': 6, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': False, 'addtl_experiment': False, 'experiment_test':0 }
EX3 ={'region_name':region_name, 'num_lags_obs_RZSM': 9, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': False, 'addtl_experiment': False, 'experiment_test':0 }
EX4 ={'region_name':region_name, 'num_lags_obs_RZSM': 12, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': False, 'addtl_experiment': False, 'experiment_test':0 }


#Include all predictors for obs but DO include the current reforecast RZSM 
EX14 ={'region_name':region_name, 'num_lags_obs_RZSM': 3, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': False, 'addtl_experiment': True, 'experiment_test':1 }
EX15 ={'region_name':region_name, 'num_lags_obs_RZSM': 6, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': False, 'addtl_experiment': True, 'experiment_test':2 }
EX16 ={'region_name':region_name, 'num_lags_obs_RZSM': 9, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': False, 'addtl_experiment': False, 'experiment_test':3 }
EX17 ={'region_name':region_name, 'num_lags_obs_RZSM': 12, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': False, 'addtl_experiment': False, 'experiment_test':4 }


In [17]:
#Include only OBS RZSM predictors and the current reforecast RZSM week 

EX5 ={'region_name':region_name, 'num_lags_obs_RZSM': 3, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }
EX6 ={'region_name':region_name, 'num_lags_obs_RZSM': 6, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }
EX7 ={'region_name':region_name, 'num_lags_obs_RZSM': 9, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }
EX8 ={'region_name':region_name, 'num_lags_obs_RZSM': 12, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }

#Include only OBS RZSM predictors, previous weeks prediction, and current reforecast RZSM week 
EX18 ={'region_name':region_name, 'num_lags_obs_RZSM': 3, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': True, 'experiment_test':1 }
EX19 ={'region_name':region_name, 'num_lags_obs_RZSM': 6, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': True, 'experiment_test':2 }
EX20 ={'region_name':region_name, 'num_lags_obs_RZSM': 9, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': True, 'experiment_test':3 }
EX21 ={'region_name':region_name, 'num_lags_obs_RZSM': 12, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': True, 'experiment_test':4 }

#Include only OBS RZSM predictors DO NOT INCLUDE ANY REFORECAST OR PREDICTION
EX22 ={'region_name':region_name, 'num_lags_obs_RZSM': 3, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': True, 'experiment_test':5 }
EX23 ={'region_name':region_name, 'num_lags_obs_RZSM': 6, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': True, 'experiment_test':6 }
EX24 ={'region_name':region_name, 'num_lags_obs_RZSM': 9, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': True, 'experiment_test':7 }
EX25 ={'region_name':region_name, 'num_lags_obs_RZSM': 12, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': True, 'experiment_test':8 }


In [18]:
#Include only OBS RZSM predictors, OBS other variables, previous predictions of RZSM, and current reforecast lead

EX9 ={'region_name':region_name, 'num_lags_obs_RZSM': 3, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }
EX10 ={'region_name':region_name, 'num_lags_obs_RZSM': 6, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }
EX11 ={'region_name':region_name, 'num_lags_obs_RZSM': 9, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }
EX12 ={'region_name':region_name, 'num_lags_obs_RZSM': 12, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }


In [19]:

# EX26 ={'region_name':region_name, 'num_lags_obs_RZSM': 3, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }

# run_EXPERIMENT(num_lags_obs_RZSM=0,include_lags_obs_pwat_spfh_tmax=False,include_reforecast_or_not=True, addtl_experiment = False, experiment_test = 26)

In [20]:
#This like EX9 and EX10 except now we are adding additional reforecast predictors from week 1 and 2

EX27 ={'region_name':region_name, 'num_lags_obs_RZSM': 3, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':1 }
EX28 ={'region_name':region_name, 'num_lags_obs_RZSM': 6, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':1 }



In [21]:
def reshape_to_original(file,train_val_test):
    if train_val_test == 'train':
        return(np.reshape(file,(file.shape[0]//11, file.shape[0]//835, file.shape[1],file.shape[2],file.shape[3])))
    else:
        return(np.reshape(file,(file.shape[0]//11, file.shape[0]//104, file.shape[1],file.shape[2],file.shape[3])))

In [22]:
def reshape_OBS_to_original(file,train_val_test):
    if train_val_test == 'train':
        return(np.reshape(file,(file.shape[0]//11, file.shape[0]//835, file.shape[1],file.shape[2])))
    else:
        return(np.reshape(file,(file.shape[0]//11, file.shape[0]//104, file.shape[1],file.shape[2])))

In [23]:
# def convert_to_dataframe_for_input(training_input,validation_input,testing_input):
#     '''This would be needed if we are tyring to compute XGBoost over the entire CONUS'''
#     training_input.shape
#     #First we want to convert back to the original format (training is 835,11,48,96, N)

#     train_out = np.zeros(shape=(835*48*96,training_input.shape[-1]))

#     #Training
#     row_start = 0
#     for date in range(training_input.shape[0]):
#         for Y in range(training_input.shape[2]):
#             for X in range(training_input.shape[3]):
#                 # break
#                 data_channels = training_input[date,0,Y,X,:].flatten()
#                 train_out[row_start,:] = data_channels
#                 row_start +=1

#         train_out = np.zeros(shape=(835*48*96,training_input.shape[-1]))

#     #Validation
#     val_out = np.zeros(shape=(104*48*96,validation_input.shape[-1]))
    
#     row_start = 0
#     for date in range(validation_input.shape[0]):
#         for Y in range(validation_input.shape[2]):
#             for X in range(validation_input.shape[3]):
#                 # break
#                 data_channels = validation_input[date,0,Y,X,:].flatten()
#                 val_out[row_start,:] = data_channels
#                 row_start +=1


#     #Validation
#     test_out = np.zeros(shape=(104*48*96,testing_input.shape[-1]))
    
#     row_start = 0
#     for date in range(testing_input.shape[0]):
#         for Y in range(testing_input.shape[2]):
#             for X in range(testing_input.shape[3]):
#                 # break
#                 data_channels = testing_input[date,0,Y,X,:].flatten()
#                 val_out[row_start,:] = data_channels
#                 row_start +=1
    
#     return(train_out,val_out,test_out)

In [24]:
def train_model_for_cell(i_Y, i_X, x_train_full, x_val_full, x_test_full, y_train_full, y_val_full, y_test_full, sim_channel_list):

    params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'max_depth': 8,
        'eta': 0.1,
        'verbosity': 0,  # This silences the printing
        'n_jobs': num_cores,
    }
    
    space = {
        'max_depth': hp.quniform('max_depth', 4, 10, 1),
        'eta': hp.uniform('eta', 0.01, 0.3),
        'subsample': hp.uniform('subsample', 0.5, 1),
        'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    }
        
    
    # Prepare data slices based on i_Y and i_X
    x_train = x_train_full[:, i_Y, i_X, :]
    x_val = x_val_full[:, i_Y, i_X, :]
    x_test = x_test_full[:, i_Y, i_X, :]
    
    y_train = y_train_full[:, i_Y, i_X]
    y_val = y_val_full[:, i_Y, i_X]
    y_test = y_test_full[:, i_Y, i_X]

    # Prepare DMatrix objects
    dtrain = xgb.DMatrix(x_train, label=y_train)
    dval = xgb.DMatrix(x_val, label=y_val)

    # Hyperparameter tuning setup
    trials = Trials()
    best_hyperparams = fmin(fn=lambda params: XGBoost_run(params, dtrain, dval),
                            space=space,
                            algo=tpe.suggest,
                            max_evals=50,
                            trials=trials)
    best_hyperparams['max_depth'] = int(best_hyperparams.get('max_depth', 6))

    # Update model parameters with best_hyperparams
    params.update(best_hyperparams)

    # Train model with best hyperparameters
    bst = xgb.train(params, dtrain, num_boost_round=500, evals=[(dtrain, 'train'), (dval, 'val')],
                    early_stopping_rounds=30, verbose_eval=False)

    # Make predictions
    prediction_output_test = bst.predict(xgb.DMatrix(x_test), iteration_range=(0, bst.best_iteration + 1))
    prediction_output_val = bst.predict(xgb.DMatrix(x_val), iteration_range=(0, bst.best_iteration + 1))
    prediction_output_train = bst.predict(xgb.DMatrix(x_train), iteration_range=(0, bst.best_iteration + 1))


    # Assuming bst is your trained XGBoost model
    feature_importance = bst.get_score(importance_type='weight')  # You can also use 'gain' or 'cover'
    # print(feature_importance)

    #Now replace the feature importance labels with the actual channels
    out_dict = {}
    for idx,(k,v) in enumerate(feature_importance.items()):
        # break
        out_dict[sim_channel_list[idx]] = v
        
    #Now save into a dataset
    feature_importance_out = json.dumps(out_dict)
    
    # Construct result dictionary
    results_dict = {
        'test_prediction': prediction_output_test,
        'validation_prediction': prediction_output_val,
        'training_prediction': prediction_output_train,
        'i_Y': i_Y,
        'i_X': i_X,
        'feature_importance':feature_importance_out,
        # Add feature importance or other results as needed
    }

    return results_dict


In [25]:
# def train_model_for_cell(i_Y, i_X):
#     # This function would contain the code to train a model for a single grid cell
#     # It should return whatever results you need, such as predictions or performance metrics

#     #For hyperparameter tuning
    # space = {
    #     'max_depth': hp.quniform('max_depth', 4, 10, 1),
    #     'eta': hp.uniform('eta', 0.01, 0.3),
    #     'subsample': hp.uniform('subsample', 0.5, 1),
    #     'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    # }
    
#     # Specify model parameters
    # params = {
    #     'objective': 'reg:squarederror',
    #     'eval_metric': 'rmse',
    #     'max_depth': 8,
    #     'eta': 0.1,
    #     'verbosity': 0,  # This silences the printing
    #     'n_jobs': num_cores,
    # }
#     # Number of boosting rounds (or trees)
#     num_boost_round = 500

#     x_train = x_train_full[:,i_Y,i_X,:] 
#     x_val = x_val_full[:,i_Y,i_X,:]
#     x_test = x_test_full[:,i_Y,i_X,:]
#     x_train.shape

#     #For obs
#     y_train = y_train_full[:,i_Y,i_X,:]
#     y_val = y_val_full[:,i_Y,i_X,:]
#     y_test = y_test_full[:,i_Y,i_X,:]

#     dtrain = xgb.DMatrix(x_train, label=y_train)
#     dval = xgb.DMatrix(x_val, label=y_val)
#     # dtest = xgb.DMatrix(x_test, label=y_test)

#     trials = Trials()
#     best_hyperparams = fmin(fn=XGBoost_run, space=space, algo=tpe.suggest, max_evals=50, trials=trials)
#     best_hyperparams['max_depth'] = int(best_hyperparams.get('max_depth', 6))  # Default to 6 if not set

#     # Specify model parameters
#     params = {
#         'objective': 'reg:squarederror',
#         'eval_metric': 'rmse',
#         'verbosity': 0,  # This silences the printing
#         **best_hyperparams,  # Unpack the best hyperparameters here
#         'n_jobs': num_cores,
#     }
    
#     # Train model
#     bst = xgb.train(params, dtrain, num_boost_round, [(dtrain, 'train'), (dval, 'val')], early_stopping_rounds=30, verbose_eval=False)
#     # Make predictions

    
#     #Make predictions
#     prediction_output_test = bst.predict(xgb.DMatrix(x_test),iteration_range=(0, bst.best_iteration + 1))
#     prediction_output_val = bst.predict(xgb.DMatrix(x_val),iteration_range=(0, bst.best_iteration + 1))
#     prediction_output_train = bst.predict(xgb.DMatrix(x_train),iteration_range=(0, bst.best_iteration + 1))

#     # Assuming bst is your trained XGBoost model
#     feature_importance = bst.get_score(importance_type='weight')  # You can also use 'gain' or 'cover'
#     # print(feature_importance)

#     #Now replace the feature importance labels with the actual channels
#     out_dict = {}
#     for idx,(k,v) in enumerate(feature_importance.items()):
#         # break
#         out_dict[sim_channel_list[idx]] = v
        
#     #Now save into a dataset
#     feature_importance_out = json.dumps(out_dict)

#     results_dict = {
#         'test_prediction':prediction_output_test,
#         'validation_prediction':prediction_output_val,
#         'training_prediction':prediction_output_train,
#         'feature_importance':feature_importance_out,
#         'i_Y':i_Y,
#         'i_X':i_X,
#     }

#     return(results_dict)

In [26]:
def XGBoost_run(params, dtrain, dval):
    # Extract parameters from the params argument
    max_depth = int(params['max_depth'])
    eta = params['eta']
    subsample = params['subsample']
    colsample_bytree = params['colsample_bytree']

    # Add other fixed parameters here
    fixed_params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'verbosity': 0,
    }
    
    # Combine both fixed and tunable parameters
    all_params = {**fixed_params, 'max_depth': max_depth, 'eta': eta, 'subsample': subsample, 'colsample_bytree': colsample_bytree}
    
    # Initialize variables for storing best scores (or use a more appropriate metric according to your needs)
    best_score = float('inf')
    
    # Assuming training_input, validation_input, etc., are globally available or passed in some other way
    # For each grid cell, let's run an XGBoost model

    # x_train = bn.nanmean(training_input[:,:,i_Y,i_X,:], axis=1)
    # y_train = np.expand_dims(obs_final_train[:,0,i_Y,i_X], -1)
    # dtrain = xgb.DMatrix(x_train, label=y_train)
    
    # # Repeat for validation data
    # x_val = bn.nanmean(validation_input[:,:,i_Y,i_X,:], axis=1)
    # y_val = np.expand_dims(obs_final_validation[:,0,i_Y,i_X], -1)
    # dval = xgb.DMatrix(x_val, label=y_val)

    # Train the model
    bst = xgb.train(all_params, dtrain, num_boost_round=500, evals=[(dval, 'val')], early_stopping_rounds=30, verbose_eval=False)
    
    # Update best_score if necessary (here using early stopping's best score)
    if bst.best_score < best_score:
        best_score = bst.best_score

    return {'loss': best_score, 'status': STATUS_OK}

In [27]:
def load_observations_and_model_inputs_and_RUN(lead, include_reforecast_or_not, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, addtl_experiment, experiment_test, region_name):

        
    
    day_num = (lead_week * 7) - 1

    print('Opening a template dataset to add predictions and feature importance')
    global template
    template = xr.open_mfdataset(f'{source}/GEFSv12_reforecast/soilw_bgrnd/baseline_RZSM_anomaly/*.n*',combine='nested',concat_dim=['S']).sel(L=[day_num])
    temp_out = template.copy(deep=True)
    temp_out = temp_out.isel(S=0).expand_dims({'S':1})
    
    # Assuming obs_RZSM_percentile is an xarray Dataset
    if 'feature_importance' not in temp_out:
        temp_out['feature_importance'] = xr.DataArray(np.empty(temp_out.RZSM.shape, dtype='object'), dims=template.RZSM.dims)
    
    del temp_out['RZSM']
    
    temp_out = temp_out.isel(S=0,M=0,L=0)
    
    
    # print(addtl_experiment)
    #Training data
    #Needed for calculating the loss for individual UNET predictions (each has 4 predictions for each variable)
    def return_training_verification_data(RZSM_or_tmax):
        #We had multiple verification inputs in the past, but now we just reduced to RZSM
        if RZSM_or_tmax == 'RZSM':
            return(np.array(tf.expand_dims(obs_final_train,-1)))

    
    #Validation data
    def return_validation_verification_data(RZSM_or_tmax):
        #We had multiple verification inputs in the past, but now we just reduced to RZSM
        if RZSM_or_tmax == 'RZSM':
            return(np.array(tf.expand_dims(obs_final_validation,-1)))

    
    
    def print_shape(file,name):
        print(f'Shape of {name} is {file.shape}')

    print('Loading parameters such as experiment name, lag integer list, etc. ')
    save_experiment_name, experiment_name_out, experiment_name,lag_integer_list, \
    channel_save_dir, training_input_file, validation_input_file, \
    testing_input_file,checkpoint_filepath, losses_dir,save_checkpoint_dir, \
    save_experiment_dir = return_experiment_name_and_lags_and_channel_dir(include_reforecast_or_not, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, addtl_experiment, experiment_test, region_name)

    print(f'Working on experiment {experiment_name_out}')

    with open(f'{channel_save_dir}/{save_experiment_name}_channel_list.txt', 'r') as file:
        # Read all lines from the file into a list
        channel_list = file.readlines()

    global obs_final_train,obs_final_validation,obs_final_testing

    obs_final_train,obs_final_validation,obs_final_testing = f.load_verification_observations_updated(lead,verification_directory)

    
    
    obs_final_train = reshape_OBS_to_original(obs_final_train,'train')
    obs_final_train.shape
    obs_final_validation = reshape_OBS_to_original(obs_final_validation,'validation')
    obs_final_validation.shape
    obs_final_testing = reshape_OBS_to_original(obs_final_testing,'test')
    obs_final_testing.shape
    
    global training_input,validation_input,testing_input
    
    training_input = reshape_to_original(np.load(training_input_file),'train')
    training_input.shape
    validation_input = reshape_to_original(np.load(validation_input_file),'validation')
    testing_input = reshape_to_original(np.load(testing_input_file),'testing')

    final_prediction_output_test = np.empty(shape=(testing_input.shape[0],testing_input.shape[2],testing_input.shape[3]))
    final_prediction_output_test.shape
    final_prediction_output_val = np.empty(shape=(validation_input.shape[0],validation_input.shape[2],validation_input.shape[3]))
    final_prediction_output_train = np.empty(shape=(training_input.shape[0],training_input.shape[2],training_input.shape[3]))


    print('Finalizing the datasets as inputs into the algorithm')

    global x_train_full, x_val_full, x_test_full
    
    x_train_full = bn.nanmean(training_input,axis=1) #We only want the ensemble mean (mainly because the observations are repeated for each ensemble member)
    x_val_full = bn.nanmean(validation_input,axis=1)
    x_test_full = bn.nanmean(testing_input,axis=1)
    x_train_full.shape

    #For obs
    global y_train_full, y_val_full, y_test_full
    
    y_train_full =  bn.nanmean(obs_final_train,axis=1)
    y_val_full =  bn.nanmean(obs_final_validation,axis=1)
    y_test_full =  bn.nanmean(obs_final_testing,axis=1)
    
    y_train_full = np.expand_dims(y_train_full,-1)
    y_train_full.shape
    y_val_full = np.expand_dims(y_val_full,-1)
    y_test_full = np.expand_dims(y_test_full,-1)
    
    cell_indices = [(i_Y, i_X) for i_Y in range(training_input.shape[2]) for i_X in range(training_input.shape[3])]

    # #small test
    # cell_indices = [(i_Y, i_X) for i_Y in range(2) for i_X in range(2)]

    global sim_channel_list
    sim_channel_list = [i.split('\n')[0].split('lag ')[-1] for i in channel_list]

    print('\n Starting multiprocessing algorithm.')
    # Use a ProcessPoolExecutor to parallelize the training
    # with ProcessPoolExecutor(max_workers=num_cores) as executor:
    #     futures = [executor.submit(train_model_for_cell, i_Y, i_X) for i_Y, i_X in cell_indices]

    with ProcessPoolExecutor(max_workers=num_cores) as executor:
        futures = [
            executor.submit(
                train_model_for_cell, 
                i_Y, i_X, 
                x_train_full, x_val_full, x_test_full, 
                y_train_full, y_val_full, y_test_full, sim_channel_list
            ) for i_Y, i_X in cell_indices
        ]

        # Initialize a counter
        completed_count = 0
        total_tasks = len(futures)
        
        # As each future completes, do something with its result
        for future in concurrent.futures.as_completed(futures):
            # break
            result = future.result()

            final_prediction_output_test[:,result['i_Y'],result['i_X']] = result['test_prediction']
            final_prediction_output_val[:,result['i_Y'],result['i_X']] = result['validation_prediction']
            final_prediction_output_train[:,result['i_Y'],result['i_X']] = result['training_prediction']
            
            temp_out['feature_importance'][result['i_Y'],result['i_X']] = result['feature_importance']
            
            completed_count += 1
            print(f"Completed {completed_count} out of {total_tasks} tasks.")
    #Now save the final output

    train_val_dir = f'predictions_XGBOOST/{region_name}/Wk{lead}_training_validation'
    test_dir = f'predictions_XGBOOST/{region_name}/Wk{lead}_testing'

    os.system(f'mkdir -p {test_dir} {train_val_dir}')
    
    # predictions.shape
    np.save(f'{test_dir}/Wk{lead}_testing_{experiment_name_out}.npy',final_prediction_output_test)

    np.save(f'{train_val_dir}/Wk{lead}_training_{experiment_name_out}.npy',final_prediction_output_train)

    np.save(f'{train_val_dir}/Wk{lead}_validation_{experiment_name_out}.npy',final_prediction_output_val)

    save_feature_importance = f'xgboost_feature_importance/{region_name}/Wk{lead}'
    os.system(f'mkdir -p {save_feature_importance}')

    save_name = f'{save_feature_importance}/{experiment_name_out}.nc'
    os.system(f'rm {save_name}') #sometimes it won't overwrite
    template.to_netcdf(f'{save_name}')

    
    return(0)

In [28]:
def run_model_and_prediction(experiment):
    global lead_week
    for lead_week in [1,2,3,4,5]:
        global lead
        lead=lead_week
    
        generate_model_inputs(region_name=experiment['region_name'], lead = lead_week, num_lags_obs_RZSM=experiment['num_lags_obs_RZSM'],
                              include_lags_obs_pwat_spfh_tmax=experiment['include_lags_obs_pwat_spfh_tmax'],include_reforecast_or_not=experiment['include_reforecast_or_not'], 
                              addtl_experiment = experiment['addtl_experiment'], experiment_test = experiment['experiment_test'])


        load_observations_and_model_inputs_and_RUN(region_name=experiment['region_name'], lead = lead_week, num_lags_obs_RZSM=experiment['num_lags_obs_RZSM'],
                              include_lags_obs_pwat_spfh_tmax=experiment['include_lags_obs_pwat_spfh_tmax'],include_reforecast_or_not=experiment['include_reforecast_or_not'], 
                              addtl_experiment = experiment['addtl_experiment'], experiment_test = experiment['experiment_test'])


In [31]:
# full_list = [EX1,EX2,EX5,EX6,EX9,EX10,EX14,EX15,EX22,EX27,EX28]
full_list = [EX28]

In [ ]:
for experiment in full_list:
    run_model_and_prediction(experiment)

In [ ]:
def load_model_make_11_predictions(experiment):
    #test 
    # experiment = EX28

    #First load the model

In [ ]:
#for testing
lead,lead_week = 1,1
experiment=EX10
experiment['region_name']
lead = lead_week
num_lags_obs_RZSM=experiment['num_lags_obs_RZSM']
include_lags_obs_pwat_spfh_tmax=experiment['include_lags_obs_pwat_spfh_tmax']
include_reforecast_or_not=experiment['include_reforecast_or_not']
addtl_experiment = experiment['addtl_experiment']
experiment_test = experiment['experiment_test']